In [2]:
import diffuserSR3.dataset as data 
import diffuserSR3.pipeline as Pipeline
import diffuserSR3.unet2D as Unet
import diffuserSR3.util as Utils
import tqdm
import torch


In [3]:
opt={"name": "Flair",
     
     "data":{
        "train_dataroot": "/share/projects/ottopia/superstable/sr3/flair/data/train_64_512",
        "test_dataroot": "/share/projects/ottopia/superstable/sr3/flair/data/test_64_512",
        "l_resolution": 64,
        "r_resolution": 512,
        "use_shuffle": True,
        "train_data_len": -1,
        "test_data_len": 50,
        
        },
     "model":{
         "image_size":512,
         "in_channels":6,
         "out_channels":3,
         "layers_per_block":5,
         "block_out_channels": (128, 128, 256, 256, 512, 512),
         "down_block_types":(
            "DownBlock2D",  # a regular ResNet downsampling block
            "DownBlock2D",
            "DownBlock2D",
            "DownBlock2D",
            "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
            "AttnDownBlock2D",# Initialize accelerator and tensorboard logging
            ),
         "up_block_types":(
            "UpBlock2D",  # a regular ResNet upsampling block
            "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
            "AttnUpBlock2D",
            "UpBlock2D",
            "UpBlock2D",
            "UpBlock2D",
            ),
         "scheduler":{
            "type":"DDIM",
             "num_train_timesteps":1000,
             "num_test_timesteps":100,
             "beta_start":0.0001,
             "beta_end":0.02,
             "beta_schedule": "linear",
             "eta": .0
            },   
         },
     
         "trainning":{
            "num_epochs":100,
            "batch_size": 2,
            "use_shuffle": True,
            "num_workers": 1
         },
         "accelerator":{
            "mixed_presition":"fp16",
            "gradient_accumulation_steps":1,
            
         }
    }
dataset = data.create_dataset(opt['data'],'train')

In [5]:
dataloadet = data.create_dataloader(dataset,opt['trainning'],"test")

In [6]:
data1=[]
for i,(batch,name) in enumerate(dataloadet):
    data1.append(batch)
    if i>1:
        break

In [7]:
opt['model']['in_channels']

6

In [11]:
data1[0].shape

torch.Size([1, 6, 512, 512])

In [7]:
model=Unet.crreate_unet2D(opt['model'])

In [8]:
# Initialize accelerator and tensorboard logging
accelerator = Utils.create_accelerator(opt['accelerator'])


/share/home/e2109121/.conda/envs/superstable/lib/python3.11/site-packages/accelerate/accelerator.py:249: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jegomezg. Use `wandb login --relogin` to force relogin


In [ ]:
# Prepare everything
# There is no specific order to remember, you just need to unpack the
# objects in the same order you gave them to the prepare method.
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)
global_step = 0

# Now you train the model

device = torch.device("cuda:0")




for epoch in range(opt['num_epochs']):

    progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
    progress_bar.set_description(f"Epoch {epoch}")
    
    for step, (batch,index) in enumerate(train_dataloader):


        scheduler = Pipeline.create_SR3scheduler()       
        noise = torch.randn(batch.shape).to(device)   
        bs = batch.shape[0]            
        timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (bs,),device=device).long()  
        noisy_images = scheduler.add_noise(data1[0],timesteps=timesteps,noise=noise)
        
        with accelerator.accumulate(model):
            # Predict the noise residual
            noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
            loss = F.mse_loss(noise_pred, noise)
            accelerator.backward(loss)
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        progress_bar.update(1)
        logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
        progress_bar.set_postfix(**logs)
        accelerator.log(logs, step=global_step)
        global_step += 1
    # After each epoch you optionally sample some demo images with evaluate() and save the model
    if accelerator.is_main_process:
        
        
        pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)
        if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:

            evaluate(config, epoch, pipeline)
            if config.push_to_hub:
                repo.push_to_hub(commit_message=f"Epoch {epoch}", blocking=True)
            else:
                pipeline.save_pretrained(config.output_dir)

In [1]:
import diffuserSR3.dataset as Data 
import diffuserSR3.pipeline as Pipeline
import diffuserSR3.unet2D as Unet
import diffuserSR3.util as Utils
import diffuserSR3.logger as Logger

from diffusers.optimization import get_cosine_schedule_with_warmup

import logging
import argparse

from tqdm import tqdm
import wandb

import torch
import torch.nn.functional as F


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('-c', '--config', type=str, default='config.json',
                        help='JSON file for configuration')
    parser.add_argument('-p', '--phase', type=str, choices=['train', 'val'],
                        help='Run either train(training) or val(generation)', default='train')
    parser.add_argument('-gpu', '--gpu_ids', type=str, default=None)
    parser.add_argument('-debug', '-d', action='store_true')


    args = parser.parse_args()
    opt = Logger.parse(args)
    # Convert to NoneDict, which return None for missing key.
    opt = Logger.dict_to_nonedict(opt)
    
    Logger.setup_logger(None, opt['path']['log'], 'train', level=logging.INFO, screen=True)
    Logger.setup_logger('val', opt['path']['log'], 'val', level=logging.INFO)

    logger = logging.getLogger('base')
    logger.info(Logger.dict2str(opt))
    
    # dataset
    if opt['phase'] == 'train' and args.phase != 'val':
        train_set = Data.create_dataset(opt['data'], opt['phase'])
        train_loader = Data.create_dataloader(train_set, opt['data'], 'train')
        val_set = Data.create_dataset(opt['data'], opt['phase'])
        val_loader = Data.create_dataloader(val_set, opt['data'], 'test')
    elif opt['phase'] == 'test':
        val_set = Data.create_dataset(opt['data'], 'test')
        val_loader = Data.create_dataloader(val_set, opt['data'], 'test')
    logger.info('Initial Dataset Finished')

/share/home/e2109121/.conda/envs/superstable/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
usage: ipykernel_launcher.py [-h] [-c CONFIG] [-p {train,val}] [-gpu GPU_IDS]
                             [-debug]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"90aa1686-61de-4e4b-adee-f761a2d0a0cb" --shell=9007 --transport="tcp" --iopub=9009 --f=/share/home/e2109121/.local/share/jupyter/runtime/kernel-v2-25798872X7CTatTyibS.json


SystemExit: 2

/share/home/e2109121/.conda/envs/superstable/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
